# Beneish M-Score Analysis

This notebook calculates the Beneish M-Score to assess the likelihood of earnings manipulation
based on financial statement indicators.


In [1]:
from src.setup import *

## What is the Beneish M-Score?

The Beneish M-Score is a forensic accounting model that uses eight financial ratios to
identify companies that are likely manipulating earnings.

Interpretation:
- **M-Score < -2.22** → Low likelihood of manipulation
- **M-Score > -2.22** → Possible earnings manipulation


## Indicators Used

1. DSRI — Days Sales in Receivables Index  
2. GMI — Gross Margin Index  
3. SGI — Sales Growth Index  
4. AQI — Asset Quality Index  
5. DEPI — Depreciation Index  
6. SGAI — SG&A Expense Index  
7. TATA — Total Accruals to Total Assets  
8. LVGI — Leverage Index  


In [2]:

def calculate_beneish_m_score(df, info, balance_sheet, ticker):
    """
    Calculate the Beneish M-Score to detect potential earnings manipulation.

    Parameters
    ----------
    df : pandas.DataFrame
        Quarterly financial dataframe with datetime index.
    info : object
        Financial data provider (e.g. yfinance.Ticker object).
    balance_sheet : pandas.DataFrame
        Balance sheet data (latest year first).
    ticker : str
        Stock ticker symbol.

    Returns
    -------
    float
        Beneish M-Score
    """
    
    # Ensure we have at least two years of data
    years = sorted(df.index.year.unique())
    if len(years) < 2:
        raise ValueError("At least two years of data are required")

    t, t1 = years[-1], years[-2]

    # --- DSRI: Days Sales in Receivables Index ---
    ar_t = df.loc[str(t), f"{ticker}: Quarterly Accounts Receivable"].iloc[-1]
    ar_t1 = df.loc[str(t1), f"{ticker}: Quarterly Accounts Receivable"].iloc[-1]

    revenue_t = info.financials.loc["Total Revenue"].iloc[0]
    revenue_t1 = info.financials.loc["Total Revenue"].iloc[1]

    dsri = (ar_t / revenue_t) / (ar_t1 / revenue_t1) if revenue_t1 != 0 and revenue_t != 0 else np.nan

    # --- GMI: Gross Margin Index ---
    gp_t = info.financials.loc["Gross Profit"].iloc[0]
    gp_t1 = info.financials.loc["Gross Profit"].iloc[1]

    gross_margin_t = gp_t / revenue_t if revenue_t != 0 else np.nan
    gross_margin_t1 = gp_t1 / revenue_t1  if revenue_t1 != 0 else np.nan
    gmi = gross_margin_t1 / gross_margin_t if gross_margin_t != 0 else np.nan

    # --- SGI: Sales Growth Index ---
    sgi = revenue_t / revenue_t1

    # --- AQI: Asset Quality Index ---
    current_assets_t = balance_sheet.loc["Current Assets"].iloc[0]
    current_assets_t1 = balance_sheet.loc["Current Assets"].iloc[1]

    ppe_t = balance_sheet.loc["Net PPE"].iloc[0]
    ppe_t1 = balance_sheet.loc["Net PPE"].iloc[1]

    total_assets_t = balance_sheet.loc["Total Assets"].iloc[0]
    total_assets_t1 = balance_sheet.loc["Total Assets"].iloc[1]

    aqi = (
        1 - ((current_assets_t + ppe_t) / total_assets_t)
    ) / (
        1 - ((current_assets_t1 + ppe_t1) / total_assets_t1)
    ) if total_assets_t != 0 and total_assets_t1 != 0 else np.nan

    # --- DEPI: Depreciation Index ---
    depreciation_t = abs(balance_sheet.loc["Accumulated Depreciation"].iloc[0])
    depreciation_t1 = abs(balance_sheet.loc["Accumulated Depreciation"].iloc[1])

    depi = (
        depreciation_t1 / (depreciation_t1 + ppe_t1)
    ) / (
        depreciation_t / (depreciation_t + ppe_t)
    ) if (depreciation_t + ppe_t) != 0 and (depreciation_t1 + ppe_t1) != 0 else np.nan


    # --- SGAI: Sales, General and Administration Index ---
    sga_t = info.financials.loc["Selling General And Administration"].iloc[0]
    sga_t1 = info.financials.loc["Selling General And Administration"].iloc[1]

    sgai = (sga_t / revenue_t) / (sga_t1 / revenue_t1) if revenue_t != 0 and revenue_t1 != 0 else np.nan

    # --- TATA: Total Accruals to Total Assets ---
    net_income_t = info.financials.loc["Net Income"].iloc[0]
    cfo_t = info.cashflow.loc["Operating Cash Flow"].iloc[0]

    tata = (net_income_t - cfo_t) / total_assets_t if total_assets_t != 0 else np.nan

    # --- LVGI: Leverage Index ---
    debt_t = balance_sheet.loc["Total Debt"].iloc[0]
    debt_t1 = balance_sheet.loc["Total Debt"].iloc[1]

    lvgi = (debt_t / total_assets_t) / (debt_t1 / total_assets_t1) if total_assets_t != 0 and total_assets_t1 != 0 else np.nan

    # --- Beneish M-Score calculation ---
    m_score = (
        -4.84
        + 0.92 * dsri
        + 0.528 * gmi
        + 0.404 * aqi
        + 0.892 * sgi
        + 0.115 * depi
        - 0.172 * sgai
        + 4.679 * tata
        - 0.327 * lvgi
    )

    return m_score


## Step 1: Calculate the Beneish M-Score


In [3]:
# Example Usage

m_score = calculate_beneish_m_score(df, info, balance_sheet, TICKER)

## Step 2: Interpret Results


In [4]:

print("\n- Beneish M-Score -\n")
print("M-Score < -2.22 → Everything is good")
print("M-Score > -2.22 → Possible manipulation\n")
print(f"M-Score: {round(m_score, 3)}")


- Beneish M-Score -

M-Score < -2.22 → Everything is good
M-Score > -2.22 → Possible manipulation

M-Score: -2.592
